<h2><center>The Captone Project - Battle of the neigborhoods <br \> <br \> Applied Data Science Capstone Course By Coursera</center></h2>

<h2><center> Identifying areas for real-estate investment </center></h2>

## Introduction: Business Problem <a name="introduction"></a>

Real-estate is one of the fastest growing markets in the United States today. The residential housing market has seen unprecendented growth in the last year, fuelled by the demands caused by pandemic (people opting to work from home, moving to cheaper cities etc). As the residential real estate market grows, commercial real estate market follows suit. The upcoming housing communities need supporting commercial establishments such as coffee shops, restaurants, office spaces, preschools etc around them. Therefore, this market presents a great avenue to invest and grow for someone who is looking for oppurtunies in real-estate market. However, these investors, indivduals or a company might not be physcially present at these locations to know where such opportunities exist. Usually, either they spend a lot of time exploring the areas on their own or send a real-estate agent to these locations. Both of which are time consuming and expensive affairs. In this project, I aim to present a data driven analysis to identify the upcoming communities and the commercial establishments around them. The goal of this project is to give a bird's eye view of the geographical location that also shows the areas with potential to grow. Using the Foursquare API I also give an idea of what commercial establishements are already in operation to further narrow down the investment opportunities.  

## Data <a name="data"></a>

For this study I picked Williamson county in Texas, which is the third fastest growing county in Texas. I did not want to pick the top most counties as those counties would already have seen enough growth. The idea is to pick the ones that have the most growth potential for the investment. Further,I looked at the fastest growing cities in Williamson county, this information is obtained from wikipedia. I used Beautiful soup to scrape wikipedia and get the information of top 5 largest cities in Williamson County. The information was from 2018 and the present day numbers might be higher. Again to ensure highest growth potential opportunites, I left the top 2 cities and picked the thrid largest city 'Georgetown' for this study. Next step was to look at where new housing communities were coming up in Georgetown.This information was obtained from Williamson county website, it had names and addresses of the communities. Once I had the addresses, I cleeaned them and used google API to obtain the location coordinates for those addresses. As the last step in "Data", I used the address coordinates in Foursqaure API to obtain the venues around them and to further explore the Georgetown city.      

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup 

#### Fastest Growing cities in Williamson County, Texas

In [2]:
url = "https://en.wikipedia.org/wiki/Williamson_County,_Texas"
html_data = requests.get(url).text

In [3]:
soup = BeautifulSoup(html_data, 'html5lib')

In [4]:
title = soup.title

In [5]:
header = soup.find("b", text="Municipal Population History")
table= header.find_parent("table")

In [6]:
table_contents = []
for row in table.find_all("tr")[2:] :
    data = {}
    data = [cell.get_text(strip=True) for cell in row.find_all("td")]
    table_contents.append(data)

In [7]:
rows = table.find_all('tr')[:2]
heading = []
for row in rows :
    hdng = [text.get_text(strip=True) for text in row.find_all("th")]
    heading = hdng

In [8]:
counties = pd.DataFrame(table_contents, columns = heading)
counties = counties.set_index('#')
counties.head()

,Largest Cities in Williamson County,1950,1960,1970,1980,1990,2000,2010,2018 (estimate)
#,,,,,,,,,
1,Round Rock,"1,438","1,878","2,811","11,812","30,923","61,136","99,887","128,739"
2,Cedar Park,202,385,692,"3,474","5,161","26,049","48,937","76,999"
3,Georgetown,"4,951","5,218","6,395","9,468","14,842","28,339","47,400","74,180"
4,Leander,-,-,-,"2,179","3,398","7,596","26,521","56,111"
5,Hutto,n/a,400,545,659,630,"1,250","14,698","26,434"


#### New home communities in Georgetown Texas

In [9]:
url1 = "https://wclife.org/new-home-communities/cities/georgetown"
url2 = "https://wclife.org/new-home-communities/cities/georgetown/P10"
html_data1 = requests.get(url1).text
html_data2 = requests.get(url2).text

In [10]:
soup1 = BeautifulSoup(html_data1, 'html5lib')
soup2 = BeautifulSoup(html_data2, 'html5lib')

#### Extracting the community names 

In [11]:
comms1 = []
for h in soup1.find_all('h2'): 
    txt = h.get_text()
    comms1.append(txt)
    
comms1 = comms1[1:-1]
comms2 = []
for h in soup2.find_all('h2'):
    txt = h.get_text()
    comms2.append(txt)
comms2 = comms2[1:-1]

In [12]:
comms = comms1 + comms2

In [13]:
print(comms)
len(comms)

['Berry Creek', 'Carlson Place', 'Cimarron Hills', 'Deer Haven', 'Georgetown Village', 'La Conterra', 'Lakeside at Lake Georgetown', 'MorningStar', 'Oaks at San Gabriel', 'Parkside at Mayfield Ranch', 'Rancho Sienna', 'SaddleCreek', 'Sun City Texas', 'Teravista by Newland Communities', 'The Brownstone at the Summit', 'Water Oak', 'Wolf Ranch', 'Woodland Park']


18

#### Extracting the community addresses

In [14]:
addrs1 = []
for p in soup1.select("p:nth-of-type(1)"):
    txt = p.get_text()
    addrs1.append(txt)
addrs1 = addrs1[0:-2]
print(addrs1)
addrs2 = []
for p in soup2.select("p:nth-of-type(1)"):
    txt = p.get_text()
    addrs2.append(txt)
addrs2 = addrs2[0:-2]
print(addrs2)

[' 30325 Tiger Woods Dr.  | Georgetown, TX  78628', ' Rockride Lane and Carlson Cove  | Georgetown, TX  78626', ' 307 Flint Ridge Trail  | Georgetown, TX  78628', ' 3816 Williams Drive  | Georgetown, TX  78633', ' Shell Road  | Georgetown, TX  78626', ' 358 Grand Junction Trl  | Georgetown, TX  78626', ' 113 Lake Spring Circle  | Georgetown, TX  78628', ' 101 Ridgeview Court  | Georgetown, TX  78628', ' 1211 Terrace Drive  | Georgetown, TX  78628', ' 101 Garner Park Cove  | Georgetown, TX  78628']
[' 404 Via De Sienna Blvd  | Georgetown, TX  78628', ' 8605 Daisy Cutter Crossing  | Georgetown, TX  78626', ' 1501 Sun City Blvd  | Georgetown, TX  78633', ' 1000 Wildflower Park Drive  | Georgetown, TX  78626', ' 204 Adams St  | Georgetown, TX  78628', ' 2609 Water Oak Parkway  | Georgetown, TX  78628', ' Wolf Ranch Parkway  | Georgetown, TX  78626', ' 101 W. Red Bud Meadow  | Georgetown, TX  78633']


In [15]:
addrs = addrs1+addrs2
len(addrs)

18

In [16]:
print(addrs)

[' 30325 Tiger Woods Dr.  | Georgetown, TX  78628', ' Rockride Lane and Carlson Cove  | Georgetown, TX  78626', ' 307 Flint Ridge Trail  | Georgetown, TX  78628', ' 3816 Williams Drive  | Georgetown, TX  78633', ' Shell Road  | Georgetown, TX  78626', ' 358 Grand Junction Trl  | Georgetown, TX  78626', ' 113 Lake Spring Circle  | Georgetown, TX  78628', ' 101 Ridgeview Court  | Georgetown, TX  78628', ' 1211 Terrace Drive  | Georgetown, TX  78628', ' 101 Garner Park Cove  | Georgetown, TX  78628', ' 404 Via De Sienna Blvd  | Georgetown, TX  78628', ' 8605 Daisy Cutter Crossing  | Georgetown, TX  78626', ' 1501 Sun City Blvd  | Georgetown, TX  78633', ' 1000 Wildflower Park Drive  | Georgetown, TX  78626', ' 204 Adams St  | Georgetown, TX  78628', ' 2609 Water Oak Parkway  | Georgetown, TX  78628', ' Wolf Ranch Parkway  | Georgetown, TX  78626', ' 101 W. Red Bud Meadow  | Georgetown, TX  78633']


In [17]:
# A dataframe of communities with corresponding addresses
ggtwn = {'Communities': comms, 'Address':addrs}
com_ggtwn = pd.DataFrame(ggtwn, columns = ['Communities', 'Address'])

In [18]:
com_ggtwn.head()

,Communities,Address
0,Berry Creek,"30325 Tiger Woods Dr. | Georgetown, TX 78628"
1,Carlson Place,"Rockride Lane and Carlson Cove | Georgetown,..."
2,Cimarron Hills,"307 Flint Ridge Trail | Georgetown, TX 78628"
3,Deer Haven,"3816 Williams Drive | Georgetown, TX 78633"
4,Georgetown Village,"Shell Road | Georgetown, TX 78626"


#### Obtaining the coordinates of the Neighborhood addresses

In [19]:
import re
import folium
from folium.features import DivIcon

In [20]:
# The code was removed by Watson Studio for sharing.

In [21]:
def get_coords(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'Georgetown, Texas, USA'
ggtwn = get_coords(api_key, address)
print('Coordinate of {}: {}'.format(address, ggtwn))

Coordinate of Georgetown, Texas, USA: [30.6332618, -97.6779842]


In [22]:
# cleaning the addresses 
cleanaddrs = []
for a in addrs:
    ca = a.replace('|', ',')
    cleanaddrs.append(ca)

In [23]:
# obtaining the coordinates using google API
lat = []
lon = []
loc = []

for a in cleanaddrs :
    l = get_coords(api_key, a)
    loc.append(l)
    lat.append(l[0])
    lon.append(l[1])

In [24]:
print(loc)

[[30.7144059, -97.66470869999999], [30.621265, -97.644688], [30.6523577, -97.7912765], [30.6729895, -97.7014293], [30.7122562, -97.68587579999999], [30.599753, -97.65625209999999], [30.6899687, -97.7290086], [30.64828319999999, -97.81913019999999], [30.6410107, -97.76340189999999], [30.5762202, -97.7716166], [30.62622959999999, -97.8184823], [30.618414, -97.6353695], [30.7165271, -97.7211944], [30.5902212, -97.66370380000001], [30.6516196, -97.68659670000001], [30.6307418, -97.7524717], [30.6372296, -97.7005088], [30.7105922, -97.7577561]]


In [25]:
len(loc)

18

In [26]:
# updated dataframe with latitude and longitude information
com_ggtwn['Address'] = cleanaddrs
com_ggtwn['latitude']= lat
com_ggtwn['longitude']=lon

In [27]:
com_ggtwn

,Communities,Address,latitude,longitude
0,Berry Creek,"30325 Tiger Woods Dr. , Georgetown, TX 78628",30.714406,-97.664709
1,Carlson Place,"Rockride Lane and Carlson Cove , Georgetown,...",30.621265,-97.644688
2,Cimarron Hills,"307 Flint Ridge Trail , Georgetown, TX 78628",30.652358,-97.791276
3,Deer Haven,"3816 Williams Drive , Georgetown, TX 78633",30.672989,-97.701429
4,Georgetown Village,"Shell Road , Georgetown, TX 78626",30.712256,-97.685876
5,La Conterra,"358 Grand Junction Trl , Georgetown, TX 78626",30.599753,-97.656252
6,Lakeside at Lake Georgetown,"113 Lake Spring Circle , Georgetown, TX 78628",30.689969,-97.729009
7,MorningStar,"101 Ridgeview Court , Georgetown, TX 78628",30.648283,-97.819130
8,Oaks at San Gabriel,"1211 Terrace Drive , Georgetown, TX 78628",30.641011,-97.763402
9,Parkside at Mayfield Ranch,"101 Garner Park Cove , Georgetown, TX 78628",30.576220,-97.771617
